In [1]:
%matplotlib inline
import sys

import arviz as az
import numpy as np
import pymc as pm

from arviz_dashboard import *

if not sys.warnoptions:
    import warnings
warnings.simplefilter("ignore")

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

## Test 1

### Create Models

In [2]:
dict_cmp = {}
d1 = az.load_arviz_data("centered_eight")
d2 = az.load_arviz_data("non_centered_eight")
dict_cmp["mA"] = d1
dict_cmp["mB"] = d2

### Build Dashboard

In [3]:
forest_dashboard = ForestDashboard(dict_cmp)
forest_dashboard.dashboard_forest()

Column
    [0] Row
        [0] MultiSelect(name='ModelSelect', options=['mA', 'mB'], value=['mA'])
    [1] FloatSlider(name='HDI Probability', step=0.05, value=0.7, width=200)
    [2] Tabs
        [0] Column
            [0] WidgetBox
                [0] Markdown(str)
                [1] Row
                    [0] Select(name='Model', options=OrderedDict([('mA', ...]), value='mA')
                    [1] Select(name='Data variable', options=OrderedDict([('mu', ...]), value='mu')
                    [2] Select(name='Coor variable')
                [2] RangeSlider(end=10, name='Rope Range', start=-10, step=1, value=(2, 5), value_end=5, value_start=2, width=200)
            [1] ParamFunction(function)
        [1] Column
            [0] Row
                [0] Checkbox(name='Ridgeplot Truncate')
            [1] Row
                [0] RangeSlider(name='Ridgeplot Quantiles', step=0.01, value=(0.25, 0.75), value_end=0.75, value_start=0.25, width=200)
                [1] FloatSlider(name='Ridgeplot Overlap', step=0.05, value=0.7, width=200)
            [2] ParamFunction(function)

## Test2

### Create Models

In [4]:
y_obs = np.random.normal(0, 1, size=100)
idatas_cmp = {}

with pm.Model() as mA:
    σ = pm.HalfNormal("σ", 1)
    y = pm.SkewNormal("y", 0, σ, observed=y_obs)
    idataA = pm.sample()
    idataA.extend(pm.sample_posterior_predictive(idataA))

with pm.Model() as mB:
    σ = pm.HalfNormal("σ", 1)
    y = pm.Normal("y", 0, σ, observed=y_obs)
    idataB = pm.sample()
    idataB.extend(pm.sample_posterior_predictive(idataB))


with pm.Model() as mC:
    μ = pm.Normal("μ", 0, 1)
    σ = pm.HalfNormal("σ", 1)
    y = pm.Normal("y", μ, σ, observed=y_obs)
    idataC = pm.sample()
    idataC.extend(pm.sample_posterior_predictive(idataC))

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [σ]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 21 seconds.
Sampling: [y]


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [σ]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 21 seconds.
Sampling: [y]


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [μ, σ]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 20 seconds.
Sampling: [y]


In [5]:
idatas_cmp["mA"] = idataA
idatas_cmp["mB"] = idataB
idatas_cmp["mC"] = idataC

### Build Dashboard

In [6]:
forest_dashboard = ForestDashboard(idatas_cmp)
forest_dashboard.dashboard_forest()

Column
    [0] Row
        [0] MultiSelect(name='ModelSelect', options=['mA', 'mB', 'mC'], value=['mA'])
    [1] FloatSlider(name='HDI Probability', step=0.05, value=0.7, width=200)
    [2] Tabs
        [0] Column
            [0] WidgetBox
                [0] Markdown(str)
                [1] Row
                    [0] Select(name='Model', options=OrderedDict([('mA', ...]), value='mA')
                    [1] Select(name='Data variable', options=OrderedDict([('σ', 'σ')]), value='σ')
                    [2] Select(name='Coor variable')
                [2] RangeSlider(end=10, name='Rope Range', start=-10, step=1, value=(2, 5), value_end=5, value_start=2, width=200)
            [1] ParamFunction(function)
        [1] Column
            [0] Row
                [0] Checkbox(name='Ridgeplot Truncate')
            [1] Row
                [0] RangeSlider(name='Ridgeplot Quantiles', step=0.01, value=(0.25, 0.75), value_end=0.75, value_start=0.25, width=200)
                [1] FloatSlider(name='Ridgeplot Overlap', step=0.05, value=0.7, width=200)
            [2] ParamFunction(function)